# ==== INTERACTIVE CLUSTERING COMPARATIVE STUDY ====
> ### Drafts.

------------------------------
## READ-ME BEFORE RUNNING

### Quick Description

***#TODO:***

### Description each steps

***#TODO:***

------------------------------
## [DRAFT] Main effects and Post-hoc analyses in Python

***WARNING***: The notebook `3_Analyze_main_effecets_and_post_hoc.ipynb` is more suitable to perform Main effects and Post-hoc analyses in **R**.

Import Python dependencies.

In [ ]:
from statsmodels.stats.anova import (
    AnovaRM,
)  # Repeated Measure ANOVA. cf. https://www.statology.org/repeated-measures-anova-python/
from statsmodels.stats.multicomp import pairwise_tukeyhsd  # Posthoc with Tuckey HSD.

Load the synthetized results.

In [ ]:
# Load synthesis dataframe.
import pandas as pd

df_analysis = pd.read_csv(
    filepath_or_buffer="../experiments/_SYNTHESIS.csv",
    sep=";",
    index_col=0,
)
df_analysis.head()

Apply Main Effect with a Repeated Measures ANOVA analysis.

In [ ]:
# Fit RM ANOVA model.
anova_results = AnovaRM(
    data=df_analysis,
    depvar="V080v__iteration",  # Metric= to analyse.
    within=[
        "preprocessing",
        "vectorization",
        "sampling",
        "clustering",
    ],  # Parameters to compare.
    subject="random_seed",  # Random seed as observation error.
).fit()

# Add symbols of significance.
anova_results.anova_table["significance"] = anova_results.anova_table.apply(
    lambda row: (
        "***"
        if row["Pr > F"] <= 0.001
        else "**"
        if row["Pr > F"] <= 0.01
        else "*"
        if row["Pr > F"] <= 0.05
        else "*"
        if row["Pr > F"] <= 0.1
        else ""
    ),
    axis=1,
)

# Add null hypothesis rejection.
anova_results.anova_table["rejected"] = anova_results.anova_table.apply(
    lambda row: (row["Pr > F"] <= 0.05), axis=1
)
anova_results.anova_table.astype({"rejected": "bool"})

# Print ANOVA analysis results
anova_results.anova_table

Apply post-hoc analysis with Tukey HSD.

In [ ]:
# For each parameter in ANOVA table.
for parameters, anova_values in anova_results.anova_table.iterrows():
    print("#############################")
    print(parameters)

    # If null hypothesis rejected...
    if anova_values["rejected"] is True:

        # Complete data with combination of parameters.
        if ":" in parameters:  # Case of a combination.
            df_analysis[parameters] = df_analysis[parameters.split(":")[0]]
            for parameter in parameters.split(":")[1:]:
                df_analysis[parameters] += ":" + df_analysis[parameter]

        # Compute post-hoc comparison.
        tukey = pairwise_tukeyhsd(
            endog=df_analysis["V080v__iteration"],
            groups=df_analysis[parameters],
            alpha=0.05,
        )
        print(tukey.summary())

***WARNING***: _The post-hoc analysis in Python are not adjust with the Tukey HSD, so results can differ from R analysis._